In [ ]:
import dask
import dask.distributed
import dask_jobqueue
import pathlib
import xarray as xr

from crims2s.distribution import fit_normal_xarray
from crims2s.util import fix_dataset_dims, add_biweekly_dim

In [ ]:
TRAINING_INPUT = '***BASEDIR***training-input'

In [ ]:
train_input_path = pathlib.Path(TRAINING_INPUT)

In [ ]:
files_list = []
for f in list(train_input_path.iterdir()):
    if 'ncep' in f.name and ('-t2m-' in f.name or '-tp-' in f.name):
        files_list.append(f)
files_list = sorted(files_list)

In [ ]:
len(files_list)

In [ ]:
sample = xr.open_dataset(files_list[0])

In [ ]:
sample.forecast_time

In [ ]:
sample.isel(lead_time=0, realization=0, forecast_time=0).t2m.plot(size=10)

In [ ]:
sample = add_biweekly_dim(xr.open_dataset(files_list[0]), weeks_12=False)

In [ ]:
sample.isnull().sum(dim=['realization', 'forecast_time', 'latitude', 'longitude'])

In [ ]:
sample.lead_time

In [ ]:
parameters = fit_normal_xarray(sample.t2m, dim=['lead_time', 'realization'])

In [ ]:
parameters

In [ ]:
parameters.isel(forecast_time=0, biweekly_forecast=0).t2m_sigma.plot()

In [ ]:
sample.isel(biweekly_forecast=1, lead_time=5, realization=0, forecast_time=0).t2m.plot()

In [ ]:
cluster = dask_jobqueue.SLURMCluster(
    env_extra=['source ***HOME***.bash_profile','conda activate s2s'],
    name='s2s',
)

In [ ]:
cluster.scale(jobs=2)

In [ ]:
client = dask.distributed.Client(cluster)

In [ ]:
client

In [ ]:
d = xr.open_mfdataset(files_list, preprocess=fix_dataset_dims)

In [ ]:
d = d.persist()

In [ ]:
d.forecast_time

In [ ]:
len(d.forecast_monthday)

In [ ]:
for md in d.forecast_monthday:
    md= str(md.data)
    month = int(md[:2])
    day = int(md[2:])
    print(f'({month}, {day}),')

In [ ]:
nan_counts = d.tp.isnull().mean(dim=['forecast_year', 'forecast_monthday', 'lead_time', 'latitude', 'longitude']).compute()

In [ ]:
nan_counts

In [ ]:
d.isnull().mean().compute()

In [ ]:
d.interpolate_na(dim='realization').isnull().mean().compute()

In [ ]:
n_null = d.tp.isnull().sum(dim=['realization']).persist()

In [ ]:
(n_null >= 1).mean().compute()

In [ ]:
(n_null >= 2).mean().compute()

In [ ]:
(n_null >= 3).mean().compute()

In [ ]:
(n_null == 4).mean().compute()

In [ ]:
nan_counts.compute()